In [1]:
pip install catboost

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Semester-V/Competitions/Classroom/train.csv')

test = pd.read_csv('/content/drive/MyDrive/Semester-V/Competitions/Classroom/test.csv')

In [3]:
train.head(10)

,id,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,16330,138.835938,45.453922,-0.096961,0.086516,3.082776,18.529846,7.179262,60.660346,0
1,5999,130.296875,51.969546,-0.005585,-0.295680,4.299331,23.557530,6.883942,52.337106,0
2,10228,127.328125,55.471714,0.090976,-0.444422,69.913880,73.240545,0.434744,-1.249027,0
3,2980,102.453125,44.412987,0.652927,1.101361,2.425585,15.501871,9.213629,106.097405,0
4,2472,104.921875,41.629431,0.189677,0.725700,3.275084,18.661805,7.389537,65.169692,0
5,9491,123.914062,49.306286,0.058630,0.255576,14.983278,56.100928,3.497040,10.309213,0
6,5274,96.617188,49.176042,0.539350,0.470347,0.562709,10.827096,20.937929,453.459566,0
7,13581,131.406250,53.508241,-0.125222,-0.184864,6.304348,30.230182,5.459737,31.094668,0
8,11984,137.945312,58.241323,-0.004645,-0.676629,4.801839,26.189399,6.370659,42.819915,0
9,17494,113.656250,42.059006,0.392402,0.638596,9.760033,36.338525,3.867539,14.783672,0


In [4]:
test.head(10)

,id,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve
0,15,130.960938,51.285872,0.072242,-0.584550,1.940635,13.092250,10.481509,147.756658
1,28,23.703125,41.677774,4.112599,16.687125,12.272575,39.323386,3.710673,14.110997
2,30,113.882812,45.562907,0.064720,0.243603,2.101171,13.008511,10.184925,145.275542
3,35,101.882812,44.301633,0.584880,0.722995,2.645485,19.539765,9.101964,90.419878
4,48,96.335938,43.957060,0.302221,0.693381,1.807692,16.216959,11.322476,144.368633
5,49,101.031250,41.848487,0.424008,0.905820,1.258361,14.716245,12.787493,172.524321
6,51,117.765625,42.016084,0.138450,0.694804,1.589465,15.623695,11.745828,153.332749
7,60,106.179688,48.586976,0.057190,-0.142287,1.576087,12.382966,11.755978,181.704063
8,65,98.570312,45.069509,0.349157,0.853319,1.815217,13.492664,10.800264,146.393325
9,70,117.242188,55.248126,0.274790,-0.433701,162.786789,42.191598,-1.199483,1.401099


In [5]:
train.shape, test.shape

((14318, 10), (3580, 9))

In [6]:
train['target_class'].value_counts()

0    13003
1     1315
Name: target_class, dtype: int64

In [7]:
test['target_class'] = -1 

test_ID = test['id']

df = pd.concat([train, test], axis=0)

df.drop('id', axis=1, inplace=True)

In [8]:
df.head(10)

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,138.835938,45.453922,-0.096961,0.086516,3.082776,18.529846,7.179262,60.660346,0
1,130.296875,51.969546,-0.005585,-0.295680,4.299331,23.557530,6.883942,52.337106,0
2,127.328125,55.471714,0.090976,-0.444422,69.913880,73.240545,0.434744,-1.249027,0
3,102.453125,44.412987,0.652927,1.101361,2.425585,15.501871,9.213629,106.097405,0
4,104.921875,41.629431,0.189677,0.725700,3.275084,18.661805,7.389537,65.169692,0
5,123.914062,49.306286,0.058630,0.255576,14.983278,56.100928,3.497040,10.309213,0
6,96.617188,49.176042,0.539350,0.470347,0.562709,10.827096,20.937929,453.459566,0
7,131.406250,53.508241,-0.125222,-0.184864,6.304348,30.230182,5.459737,31.094668,0
8,137.945312,58.241323,-0.004645,-0.676629,4.801839,26.189399,6.370659,42.819915,0
9,113.656250,42.059006,0.392402,0.638596,9.760033,36.338525,3.867539,14.783672,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17898 entries, 0 to 3579
Data columns (total 9 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Mean of the integrated profile                17898 non-null  float64
 1   Standard deviation of the integrated profile  17898 non-null  float64
 2   Excess kurtosis of the integrated profile     17898 non-null  float64
 3   Skewness of the integrated profile            17898 non-null  float64
 4   Mean of the DM-SNR curve                      17898 non-null  float64
 5   Standard deviation of the DM-SNR curve        17898 non-null  float64
 6   Excess kurtosis of the DM-SNR curve           17898 non-null  float64
 7   Skewness of the DM-SNR curve                  17898 non-null  float64
 8   target_class                                  17898 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.4 MB


In [10]:
df.var()

Mean of the integrated profile                    658.073093
Standard deviation of the integrated profile       46.829241
Excess kurtosis of the integrated profile           1.132181
Skewness of the integrated profile                 38.043154
Mean of the DM-SNR curve                          868.651666
Standard deviation of the DM-SNR curve            379.103187
Excess kurtosis of the DM-SNR curve                20.304864
Skewness of the DM-SNR curve                    11345.347132
target_class                                        0.257494
dtype: float64

In [11]:
df['Skewness of the DM-SNR curve'].describe()

count    17898.000000
mean       104.857709
std        106.514540
min         -1.976976
25%         34.960504
50%         83.064556
75%        139.309331
max       1191.000837
Name: Skewness of the DM-SNR curve, dtype: float64

In [12]:
df['Skewness of the DM-SNR curve'] = np.abs(df['Skewness of the DM-SNR curve'])

In [13]:
df['Skewness of the DM-SNR curve'] = np.log(df['Skewness of the DM-SNR curve'])

In [14]:
df['Ratio1'] = df['Excess kurtosis of the integrated profile'] / df['Skewness of the integrated profile']
df['Ratio2'] = df['Excess kurtosis of the DM-SNR curve'] / df['Skewness of the DM-SNR curve']

In [15]:
df.isna().sum()

Mean of the integrated profile                  0
Standard deviation of the integrated profile    0
Excess kurtosis of the integrated profile       0
Skewness of the integrated profile              0
Mean of the DM-SNR curve                        0
Standard deviation of the DM-SNR curve          0
Excess kurtosis of the DM-SNR curve             0
Skewness of the DM-SNR curve                    0
target_class                                    0
Ratio1                                          0
Ratio2                                          0
dtype: int64

In [16]:
test = df[df['target_class'] == -1]
test = test.drop('target_class', axis=1)

train = df[df['target_class'] != -1]

In [17]:
X = train.drop('target_class', axis=1).values

y = train['target_class'].values

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22, shuffle=True, stratify=y)

In [19]:
model = CatBoostClassifier()

model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)[:, 1]

Learning rate set to 0.029182
0:	learn: 0.6344532	total: 59.9ms	remaining: 59.9s
1:	learn: 0.5769084	total: 78.7ms	remaining: 39.3s
2:	learn: 0.5262013	total: 88.9ms	remaining: 29.6s
3:	learn: 0.4838241	total: 100ms	remaining: 25s
4:	learn: 0.4462013	total: 113ms	remaining: 22.5s
5:	learn: 0.4092826	total: 128ms	remaining: 21.2s
6:	learn: 0.3797483	total: 138ms	remaining: 19.6s
7:	learn: 0.3508000	total: 148ms	remaining: 18.4s
8:	learn: 0.3251426	total: 158ms	remaining: 17.4s
9:	learn: 0.3022781	total: 167ms	remaining: 16.6s
10:	learn: 0.2809835	total: 178ms	remaining: 16s
11:	learn: 0.2599741	total: 187ms	remaining: 15.4s
12:	learn: 0.2434501	total: 201ms	remaining: 15.3s
13:	learn: 0.2291920	total: 211ms	remaining: 14.9s
14:	learn: 0.2175696	total: 221ms	remaining: 14.5s
15:	learn: 0.2035244	total: 230ms	remaining: 14.2s
16:	learn: 0.1921825	total: 240ms	remaining: 13.9s
17:	learn: 0.1819052	total: 249ms	remaining: 13.6s
18:	learn: 0.1727691	total: 259ms	remaining: 13.4s
19:	learn: 0

In [20]:
# y_pred_log = model.predict_proba(X_test)[:, 1]

# y_test = [(np.exp(x)) for x in [i for i in y_test]]

# y_pred = [(np.exp(x)) for x in [i for i in y_pred_log]]

In [21]:
score = roc_auc_score(y_test, y_pred)
print(f'ROC AUC score: {score}') 

# Catboost - 0.9844663430122664 
# XGBoost - 0.9859398915012213

ROC AUC score: 0.9852206595006601


In [22]:
# Submission File

# model = XGBClassifier()

model.fit(X, y)

y_pred = model.predict_proba(test)[:, 1]

submission = pd.DataFrame({'id':test_ID, 'target_class':y_pred})
submission.to_csv('Submission16.csv', index=False)

Learning rate set to 0.032099
0:	learn: 0.6266480	total: 10.6ms	remaining: 10.6s
1:	learn: 0.5632994	total: 21ms	remaining: 10.5s
2:	learn: 0.5079051	total: 32ms	remaining: 10.6s
3:	learn: 0.4609377	total: 42.3ms	remaining: 10.5s
4:	learn: 0.4213618	total: 52.7ms	remaining: 10.5s
5:	learn: 0.3831279	total: 63.4ms	remaining: 10.5s
6:	learn: 0.3531351	total: 74.2ms	remaining: 10.5s
7:	learn: 0.3241655	total: 85ms	remaining: 10.5s
8:	learn: 0.2982538	total: 95.7ms	remaining: 10.5s
9:	learn: 0.2755899	total: 106ms	remaining: 10.5s
10:	learn: 0.2549486	total: 117ms	remaining: 10.5s
11:	learn: 0.2357562	total: 129ms	remaining: 10.7s
12:	learn: 0.2198307	total: 140ms	remaining: 10.6s
13:	learn: 0.2061490	total: 151ms	remaining: 10.6s
14:	learn: 0.1952241	total: 161ms	remaining: 10.6s
15:	learn: 0.1819512	total: 172ms	remaining: 10.6s
16:	learn: 0.1714911	total: 183ms	remaining: 10.6s
17:	learn: 0.1619001	total: 193ms	remaining: 10.5s
18:	learn: 0.1531004	total: 207ms	remaining: 10.7s
19:	lear

In [ ]:
# Ensemble

read_1 = pd.read_csv('Submission16.csv')

read_2 = pd.read_csv('Submission17.csv')

new = (0.75 * read_1['target_class']) + (0.25 * read_2['target_class'])

file_out = pd.DataFrame({'id': read_2['id'], 'target_class':new})

file_out.to_csv('Submission18.csv', index=False)

In [ ]:
# # Submission File

# model = CatBoostClassifier()

# model.fit(X, y)

# y_pred_log = model.predict_proba(test)[:, 1]

# y_test = [(np.exp(x)) for x in [i for i in y_test]]

# y_pred = [(np.exp(x)) for x in [i for i in y_pred_log]]

# submission = pd.DataFrame({'id':test_ID, 'target_class':y_pred})
# submission.to_csv('Submission2.csv', index=False)

In [ ]:
submission.shape

(3580, 2)